In [1]:

import import_ipynb
import PropagatedEnergies as PE
import DiscModel as DM
import InjectionSpectra as IS
import DiscModelLossFunctions as DMLF
import FileWritingReading as FWR
from scipy import integrate
from scipy.interpolate import interp1d
import numpy as np

importing Jupyter notebook from PropagatedEnergies.ipynb
importing Jupyter notebook from DiscModel.ipynb
importing Jupyter notebook from GeneralFunctions.ipynb
importing Jupyter notebook from LossFunctions.ipynb
importing Jupyter notebook from DiscModelLossFunctions.ipynb
importing Jupyter notebook from InjectionSpectra.ipynb
importing Jupyter notebook from FileWritingReading.ipynb


In [2]:

def next_flux_Particle(E_array,J_Particle_flux_array, L_Particle_1Darray,E0vsE_Particle_1Darray):
    interp_flux=interp1d(E_array,J_Particle_flux_array,fill_value="extrapolate")

    L_Particle=L_Particle_1Darray
    interp_L_Particle=interp1d(E_array,L_Particle, fill_value="extrapolate")

    E0_array=E0vsE_Particle_1Darray

    J_propagated= interp_flux(E0_array)*interp_L_Particle(E0_array)/interp_L_Particle(E_array)
    return J_propagated


In [3]:
def Propagated_Particle_Flux(E_array,J_0_Particle_array,L_Particle_2Darray,E0vsE_Particle_2Darray):
    J_flux_array=[]
    for j in range(0, len(L_Particle_2Darray)):
        if j==0:
            J_flux_array.append(J_0_Particle_array)
        else:
            J_flux_array.append(next_flux_Particle(E_array,J_flux_array[j-1], L_Particle_2Darray[::-1][j-1],E0vsE_Particle_2Darray[::-1][j-1]))
           
    return np.array(J_flux_array)

def Pitch_Angle_Averaged_Propagated_Flux_Particle(E_array, i, DiscModel, E_Interpolation_array,L_Particle_array, Range_Function_Particle_2DArray,Pitch_Angle_Sample_Size,J_0_Particle):
    Pitch_Angle_Sample=np.linspace(0+0.01,1,Pitch_Angle_Sample_Size)
    Propflux=[]
    for mu in Pitch_Angle_Sample:
        E0vsE_Particle_array=PE.E_Initial_vs_E_Propagated_Array(E_array, i, DiscModel, E_Interpolation_array, Range_Function_Particle_2DArray,mu)
        PropfluxE=Propagated_Particle_Flux(E_array,J_0_Particle,L_Particle_array,E0vsE_Particle_array)
        Propflux.append(PropfluxE)
    return integrate.trapezoid(Propflux,x=Pitch_Angle_Sample,axis=0)


In [4]:
def Primary_Electron_Flux(E_array,Radial_index,DiscModelName,Tflare,muthinj,alpha,EcoE,E_Interpolation_array,Pitch_Angle_Sample_Size):

    #Read the disc model
    DiscModel=DM.readDiscModel(DiscModelName) 

    #Electron Injection Spectra:
    J_0_Electron=IS.jOEnth(E_array, Tflare, muthinj, alpha, EcoE)

    # Electron Loss Function: 
    # The loss function is calculated for the energy array E_array and the radius i at each z or N values in the disc model

    L_Electron_array = DMLF.Loss_Function_Electron_2DArray(E_array, Radial_index, DiscModel)

    #Electron Range Function
    # The Range functions are calculated for the energy array E_array and the radius i at each z or N values in the disc model
    Range_Function_Electron_2DArray = DMLF.RElectron2DArray(E_Interpolation_array, Radial_index, DiscModel)



    #Electron Propagated Flux
    # The Propagated Flux is calculated for the energy array E_array and the radius i at each z or N values in the disc model
    Propagated_Flux_Electron=Pitch_Angle_Averaged_Propagated_Flux_Particle(E_array, Radial_index, DiscModel, E_Interpolation_array,L_Electron_array, Range_Function_Electron_2DArray,Pitch_Angle_Sample_Size,J_0_Electron)
    Propagated_Flux_Electron[Propagated_Flux_Electron < 0] = 0
    FWR.Write_Propagated_Flux_to_Text_File("Propagated_Spectra_Electron",Propagated_Flux_Electron,DiscModelName, Radial_index , Tflare)

    return Propagated_Flux_Electron



def Primary_Proton_Flux(E_array,Radial_index,DiscModelName,Tflare,muthinj,alpha,EcoE,E_Interpolation_array,Pitch_Angle_Sample_Size):
 
    DiscModel=DM.readDiscModel(DiscModelName)

    #Proton Injection Spectra:
    J_0_Proton=IS.jOPnth(E_array, Tflare, muthinj, alpha, EcoE)

    # Proton Loss Function: 
    # The loss function is calculated for the energy array E_array and the radius i at each z or N values in the disc model

    L_Proton_array = DMLF.Loss_Function_Proton_2DArray(E_array, Radial_index , DiscModel)

    #Proton Range Function
    # The Range functions are calculated for the energy array E_array and the radius i at each z or N values in the disc model

    Range_Function_Proton_2DArray = DMLF.RProton2DArray(E_Interpolation_array,Radial_index, DiscModel)



    #Proton Propagated Flux
    # The Propagated Flux is calculated for the energy array E_array and the radius i at each z or N values in the disc model
    
    Propagated_Flux_Proton=Pitch_Angle_Averaged_Propagated_Flux_Particle(E_array, Radial_index, DiscModel, E_Interpolation_array,L_Proton_array, Range_Function_Proton_2DArray,Pitch_Angle_Sample_Size,J_0_Proton)
    Propagated_Flux_Proton[Propagated_Flux_Proton < 0] = 0
    FWR.Write_Propagated_Flux_to_Text_File("Propagated_Spectra_Proton",Propagated_Flux_Proton,DiscModelName,Radial_index,Tflare)
    
    return Propagated_Flux_Proton